In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
#! git clone https://github.com/neighbornixon/Titanic.git

Cloning into 'Titanic'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 2), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 81.26 KiB | 6.25 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [102]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import seaborn as sns
import matplotlib as plt
import numpy as np

In [103]:
def handle_non_numerical_data(df):
  columns = df.columns.values

  for column in columns:
    text_digit_vals = {}
    def convert_to_int(val):
      return text_digit_vals[val]

    if df[column].dtype != np.int64 and df[column].dtype != np.float64:
      column_contents = df[column].values.tolist()
      unique_elements = set(column_contents)
      x = 0
      for unique in unique_elements:
        if unique not in text_digit_vals:
          text_digit_vals[unique] = x
          x+=1

      df[column] = list(map(convert_to_int, df[column]))

  return df


In [104]:
df = pd.read_csv('Titanic/train.csv')

df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
# Create the ticket_frequency feature
df['ticket_frequency'] = df.groupby('Ticket')['Ticket'].transform('count')

# Drop the original Ticket column
df = df.drop('Ticket', axis=1)
df = df.drop('Name', axis=1)

df = handle_non_numerical_data(df)

In [105]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_frequency
0,1,0,3,0,22.0,1,0,7.2500,70,0,1
1,2,1,1,1,38.0,1,0,71.2833,2,3,1
2,3,1,3,1,26.0,0,0,7.9250,70,0,1
3,4,1,1,1,35.0,1,0,53.1000,92,0,2
4,5,0,3,0,35.0,0,0,8.0500,70,0,1


In [106]:
df.shape

(891, 11)

In [107]:
df = df.dropna()
df.shape

(714, 11)

In [108]:
y_df = df['Survived']
y = np.array(y_df)

X_df = df.drop(columns=['Survived'])
X = np.array(X_df)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [109]:
print(len(X_train[0]))
print(len(X_test[0]))

10
10


#LOG REGRESSION


In [110]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # apply scaling on training data

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [111]:
score = pipe.score(X_test, y_test)
print(f'Testing Score of: {score:.2%}')

Testing Score of: 75.42%


# SVM

In [112]:
from sklearn.svm import SVC

In [113]:
pipe2 = make_pipeline(StandardScaler(), SVC())
pipe2.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [114]:
score2 = pipe2.score(X_test, y_test)
print(f'Testing Score of: {score2:.2%}')

Testing Score of: 80.08%


# RANDOM FOREST

In [115]:
from sklearn.ensemble import RandomForestClassifier

In [116]:
pipe3 = make_pipeline(StandardScaler(), RandomForestClassifier())
pipe3.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [117]:
score3 = pipe3.score(X_test, y_test)
print(f'Testing Score of: {score3:.2%}')

Testing Score of: 79.24%
